In [1]:
import selenium.webdriver as webdriver
from webdriver_manager.chrome import ChromeDriverManager

import requests
from bs4 import BeautifulSoup

from selenium.webdriver.common.keys import Keys
import time

In [2]:
from google_trans_new import google_translator

translator = google_translator()

def translate(text, dest):
    return translator.translate(text, dest)

def detect_lang(text):
    lang = translator.detect(text)
    if type(lang) == list:
        return lang[0]
    else:
        return lang

In [3]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_cities_in_Morocco')
print('\n'.join(r.text.splitlines()[70:480]))

soup = BeautifulSoup('\n'.join(r.text.splitlines()[70:480]), 'html.parser')

morocco_cities = []
morocco_region = []
aux = []
for i in soup.find_all('a'):
    if i.get("title") != None:
        aux.append(i.get_text())

aux
region_cities = {}
compt = 0
aux_elm = 0
for i in aux:
    if compt%2 == 0:
        aux_elm = i
        compt+=1
        continue
    if i in region_cities.keys():
        region_cities[i].append(aux_elm)
    else:
        region_cities[i] = [aux_elm]
    compt+=1

<tbody><tr>
<th>Rank
</th>
<th>City
</th>
<th data-sort-type="number">Population<br />(2014 census)<sup id="cite_ref-citydefn_6-1" class="reference"><a href="#cite_note-citydefn-6">&#91;5&#93;</a></sup><sup id="cite_ref-census2014_7-0" class="reference"><a href="#cite_note-census2014-7">&#91;6&#93;</a></sup>
</th>
<th>Region
</th></tr>
<tr>
<td>1</td>
<td><a href="/wiki/Casablanca" title="Casablanca">Casablanca</a><sup id="cite_ref-8" class="reference"><a href="#cite_note-8">&#91;b&#93;</a></sup></td>
<td>3,359,818</td>
<td><a href="/wiki/Casablanca-Settat" title="Casablanca-Settat">Casablanca-Settat</a>
</td></tr>
<tr>
<td>2</td>
<td><a href="/wiki/Fez,_Morocco" title="Fez, Morocco">Fez</a><sup id="cite_ref-9" class="reference"><a href="#cite_note-9">&#91;c&#93;</a></sup></td>
<td>1,112,072</td>
<td><a href="/wiki/F%C3%A8s-Mekn%C3%A8s" title="Fès-Meknès">Fès-Meknès</a>
</td></tr>
<tr>
<td>3</td>
<td><a href="/wiki/Tangier" title="Tangier">Tangier</a><sup id="cite_ref-10" class="refere

In [4]:
list(region_cities.values())[0]
cities = []
for i in range(len(list(region_cities.values()))):
    cities += list(region_cities.values())[i]
cities

['Casablanca',
 'Mohammedia',
 'El Jadida',
 'Dar Bouazza',
 'Settat',
 'Berrechid',
 'Bouskoura',
 'Lahraouyine',
 'Ain Harrouda',
 'Benslimane',
 'Sidi Bennour',
 'Fez',
 'Meknes',
 'Taza',
 'Sefrou',
 'Azrou',
 'Tangier',
 'Tetouan',
 'Ksar El Kebir',
 'Larache',
 'Fnideq',
 'Martil',
 'Ouazzane',
 'Al Hoceima',
 "M'diq",
 'Marrakesh',
 'Safi',
 'El Kelaa Des Sraghna',
 'Ben Guerir',
 'Essaouira',
 'Youssoufia',
 'Salé',
 'Rabat',
 'Kenitra',
 'Temara',
 'Khemisset',
 'Sidi Slimane',
 'Tifelt',
 'Sidi Kacem',
 'Souk El Arbaa',
 'Skhirat',
 'Oujda',
 'Nador',
 'Berkane',
 'Taourirt',
 'Guercif',
 'Beni Ansar',
 'Agadir',
 'Aït Melloul',
 'Inezgane',
 'Dcheira El Jihadia',
 'Oulad Teima',
 'Lqliaa',
 'Taroudant',
 'Tiznit',
 'Drargua',
 'Khouribga',
 'Beni Mellal',
 'Khenifra',
 'Fquih Ben Salah',
 'Oued Zem',
 'Suq as-Sabt Awlad an-Nama',
 'Guelmim',
 'Errachidia',
 'Ouarzazate',
 'Midelt',
 'Tan-Tan']

In [5]:
def get_links(search_term, browser):
    url = "https://www.tripadvisor.fr/Search?q=dakhla&searchSessionId=C4AB24BF9ECE1EEBA9B6B2E0DA8953A01625590656894ssid&searchNearby=false&geo=479761&sid=2964B880B9064E8D96E0880B98F41AD71625609561775&blockRedirect=true&ssrc=e&rf=6"
    
    browser.get(url)
    search_box = browser.find_element_by_id("mainSearch")
    search_box.send_keys(Keys.CONTROL + "a")
    search_box.send_keys(Keys.DELETE)
    search_box.send_keys(search_term)
    search_box.submit()
    time.sleep(3)
    try:
        links = browser.find_elements_by_class_name("result-content-columns")
    except:
        links = browser.find_elements_by_class_name("result-content-columns")
    
    results = []
    
    for link in links:
        result = link.get_attribute("onclick")
        result = result.split(',')
        href = result[3].strip()[1:-1]
        href = "https://www.tripadvisor.fr" + href
        results.append(href)
    
    return results

In [6]:
def get_infos(cities, browser):
    
    restaurants = []
    cities_restaurants = {}

    for city in cities:
        links = get_links(city, browser)
        j = 0
        for link in links:
            if j >= 5:
                    break
            url = link
            browser.get(url)

            restaurant = {}

            restaurant["name"] = None
            if len(browser.find_elements_by_xpath("/html/body/div[2]/div[1]/div/div[3]/div/div/div[1]/h1")) > 0:
                restaurant["name"] = browser.find_elements_by_xpath("/html/body/div[2]/div[1]/div/div[3]/div/div/div[1]/h1")[0].text

            restaurant["rating"] = None
            if len(browser.find_elements_by_class_name("zWXXYhVR")) > 0:
                restaurant["rating"] = browser.find_elements_by_class_name("zWXXYhVR")[0].get_attribute("title")[0:3] + "/5"
            restaurant["adresse"] = None
            if len(browser.find_elements_by_class_name("_2saB_OSe")) > 0:
                restaurant["adresse"] = browser.find_elements_by_class_name("_2saB_OSe")[0].text

            restaurant["number"] = None
            if len(browser.find_elements_by_class_name("_7c6GgQ6n")) > 1:
                restaurant["number"] = browser.find_elements_by_class_name("_7c6GgQ6n")[1].text

            restaurant["site_web"] = browser.find_elements_by_xpath("/html/body/div[2]/div[1]/div/div[3]/div/div/div[3]/span[3]/span/a")
            if len(browser.find_elements_by_xpath("/html/body/div[2]/div[1]/div/div[3]/div/div/div[3]/span[3]/span/a")) > 0:
                restaurant["site_web"] = browser.find_elements_by_xpath("/html/body/div[2]/div[1]/div/div[3]/div/div/div[3]/span[3]/span/a")[0].get_attribute("href")
            else:
                restaurant["site_web"] = None

            restaurant["cuisine"] = None
            if len(browser.find_elements_by_class_name("_1XLfiSsv")) > 0:
                restaurant["cuisine"] = browser.find_elements_by_class_name("_1XLfiSsv")[0].text

            restaurant["reviews"] = []
            i = 0
            for elm in browser.find_elements_by_class_name("noQuotes"):
                if i >= 5:
                    break
                review = translate(elm.text, "en")
                restaurant["reviews"].append(review)
                i+=1
            
            restaurant["description"] = None 
            if len(browser.find_elements_by_class_name("_1XLfiSsv")) > 1:
                restaurant["description"] = browser.find_elements_by_class_name("_1XLfiSsv")[1].text
            if len(browser.find_elements_by_class_name("_1XLfiSsv")) > 2:
                restaurant["description"] = ', ' + browser.find_elements_by_class_name("_1XLfiSsv")[2].text

            restaurants.append(restaurant)
            j+=1

        cities_restaurants[city] = restaurants
    
    return cities_restaurants

In [17]:
browser = webdriver.Chrome(ChromeDriverManager().install())
cities_restaurants = get_infos([cities[20]], browser)
browser.close()



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Reda\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=91.0.4472.124)


In [15]:
cities_restaurants["Taza"]

[]

In [16]:
cities_restaurants

{'Taza': []}

In [18]:
cities[20]

'Fnideq'